In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os, shutil, random, pickle
import cv2
import keras
from sklearn.metrics import confusion_matrix
from keras import models
from keras import layers
from keras import regularizers
from tensorflow.keras import backend as K
tf.config.set_visible_devices([], 'GPU')
tf.compat.v1.disable_eager_execution()

print("TensorFlow version: " + tf.__version__)
print("Keras version: " + keras.__version__)
print("Detected GPUs: " + str(tf.config.experimental.list_physical_devices('GPU')))
print("Detected CPUs: " + str(tf.config.experimental.list_physical_devices('CPU')))

TensorFlow version: 2.9.0
Keras version: 2.9.0
Detected GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Detected CPUs: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


### Configuration

In [2]:
train_size = 86980  # number of training images instances
valid_size = 4400  # number of validating images instances
test_size = 12630  # number of test images instances
batch_size = 20  # size of the batch
epochs = 10  # model epochs
img_rows = 32  # height of the input images
img_cols = 32  # width of the input images
output_size = 43  # number of classes (size of the one hot vector)

### Load & preprocess data

In [3]:
home = '/Users/kamil/Library/CloudStorage/OneDrive-AkademiaGórniczo-Hutniczaim.StanisławaStaszicawKrakowie/Programming/Data/tsd'
models_dir = '/Users/kamil/Library/CloudStorage/OneDrive-AkademiaGórniczo-Hutniczaim.StanisławaStaszicawKrakowie/Programming/Data/tsd/model'
model_name = 'tsd_class_v9_mac.h5'
data_dir = "data.pickle"

In [4]:
# raw data & variables
data = pd.read_pickle(os.path.join(home, data_dir))

x_train = np.flip(np.rot90(data['x_train'].transpose(0, 3, 2, 1), axes=(1, 2)), axis=1)[:train_size].astype('float64')
y_train = keras.utils.np_utils.to_categorical(data['y_train'][:train_size]).astype('float64')

x_test = np.flip(np.rot90(data['x_test'].transpose(0, 3, 2, 1), axes=(1, 2)), axis=1)[:test_size].astype('float64')
y_test = keras.utils.np_utils.to_categorical(data['y_test'][:test_size]).astype('float64')

x_valid = np.flip(np.rot90(data['x_validation'].transpose(0, 3, 2, 1), axes=(1, 2)), axis=1)[:valid_size].astype('float64')
y_valid = keras.utils.np_utils.to_categorical(data['y_validation'][:valid_size]).astype('float64')

labels = data['labels']

In [5]:
def decode_one_hot(one_hot):
    return np.where(one_hot == 1)[0][0]

In [6]:
def percentage_one_hot(one_hot):
    return round(one_hot*100, 2)

In [7]:
def show_class_example(x_data, y_data, labels, c):
    assert len(x_data) == len(y_data)
    
    f,ax = plt.subplots(1, 2)
    f.set_size_inches(10, 5)
    f_index = 0
    
    while True:
        index = random.randint(0, len(y_data))
        if decode_one_hot(y_data[index]) == c[f_index]:
            ax[f_index].set_title(labels[c[f_index]])
            ax[f_index].imshow(x_data[index])
            f_index += 1
        
        if f_index == 2:
            break

In [8]:
def show_sample(shape, x_data, y_data, labels):
    assert shape[0] > 1 and shape[1] >1
    assert len(x_data) == len(y_data)
    
    height = 4*shape[0]
    width = 4*shape[1]
    
    f, ax = plt.subplots(shape[0], shape[1])
    f.set_size_inches(width, height)
    for y in range(shape[0]):
        for x in range(shape[1]):
            index = random.randint(0, len(x_data))
            ax[y][x].set_title(labels[decode_one_hot(y_data[index])])
            ax[y][x].imshow(x_data[index])

In [ ]:
show_sample((7, 7), x_test, y_test, labels)

### Model

In [10]:
model = models.Sequential()

model.add(layers.Conv2D(64, (4, 4), activation=tf.keras.layers.PReLU(), kernel_regularizer=regularizers.L1L2(), input_shape=(img_rows, img_cols, 3), padding="valid"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(.2))

model.add(layers.Conv2D(64, (4, 4), activation=tf.keras.layers.PReLU(), kernel_regularizer=regularizers.L1L2(), padding="valid"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(.2))
model.add(layers.Flatten())

model.add(layers.Dense(output_size*2, kernel_regularizer=regularizers.L1L2(), activation='selu'))
model.add(layers.Dropout(.2))

model.add(layers.Dense(output_size*2, kernel_regularizer=regularizers.L1L2(), activation='selu'))
model.add(layers.Dropout(.2))

model.add(layers.Dense(output_size, activation='sigmoid'))

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9),
              metrics=['acc'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 29, 29, 64)        56960     
                                                                 
 batch_normalization (BatchN  (None, 29, 29, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        73344     
                                                  

In [11]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size, 
    epochs=epochs,
    #steps_per_epoch=train_size/batch_size,
    #callbacks=[
    #    tf.keras.callbacks.EarlyStopping(min_delta=0.1, patience=5)
    #],
    validation_data=(x_valid, y_valid)
)

Train on 86980 samples, validate on 4400 samples
Epoch 1/10
   20/86980 [..............................] - ETA: 6:59 - loss: 4.8947 - acc: 0.0000e+00

2023-02-16 19:14:26.116874: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-02-16 19:14:26.126350: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


86940/86980 [============================>.] - ETA: 0s - loss: 2.3323 - acc: 0.3795

/Users/kamil/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


86980/86980 [==============================] - 84s 961us/sample - loss: 2.3319 - acc: 0.3797 - val_loss: 1.0232 - val_acc: 0.7470
Epoch 2/10
86980/86980 [==============================] - 103s 1ms/sample - loss: 1.0123 - acc: 0.7171 - val_loss: 0.4784 - val_acc: 0.8980
Epoch 3/10
86980/86980 [==============================] - 106s 1ms/sample - loss: 0.6211 - acc: 0.8273 - val_loss: 0.3100 - val_acc: 0.9314
Epoch 4/10
86980/86980 [==============================] - 108s 1ms/sample - loss: 0.4500 - acc: 0.8759 - val_loss: 0.2150 - val_acc: 0.9495
Epoch 5/10
86980/86980 [==============================] - 106s 1ms/sample - loss: 0.3520 - acc: 0.9039 - val_loss: 0.1733 - val_acc: 0.9593
Epoch 6/10
86980/86980 [==============================] - 105s 1ms/sample - loss: 0.2896 - acc: 0.9210 - val_loss: 0.1549 - val_acc: 0.9616
Epoch 7/10
 2020/86980 [..............................] - ETA: 1:43 - loss: 0.2844 - acc: 0.9173

KeyboardInterrupt: 

In [ ]:
model.save(os.path.join(models_dir, model_name))

In [ ]:
model = keras.models.load_model(os.path.join(models_dir, model_name))

### Results

In [ ]:
def plot_comparison(value, val_value, metric):
    plt.clf()
    plt.figure(figsize=(25,5))
    epochs = range(len(value))
    plt.plot(epochs, value, 'bo', label=f"Training {metric}")
    plt.plot(epochs, val_value, 'b', label=f"Validation {metric}")
    plt.title(f"Comparison of Training and Validation {metric}")
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

In [ ]:
def print_evaluation(model, x_test, y_test):
    loss, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)
    print(f"Test accuracy: {round(acc * 100, 2)}%")
    print(f"Test loss: {round(loss, 2)}")

In [ ]:
def print_confusion_matrix(model, x_test, y_test, labels):
    y_pred = model.predict(x_test, verbose=0, batch_size=batch_size)
    y_pred = np.argmax(y_pred, axis=1)
    y_test = np.argmax(y_test, axis=1)
    
    cm = confusion_matrix(y_test, y_pred)
    ticks=np.linspace(0, output_size, num=output_size+1)
    plt.clf()
    plt.figure(figsize=(15, 15))
    plt.imshow(cm, interpolation='none')
    plt.colorbar()
    plt.xticks(ticks, fontsize=6)
    plt.xlabel("Predicted")
    plt.yticks(ticks, fontsize=6)
    plt.ylabel("True")
    plt.grid(True)
    plt.show()

In [ ]:
print_evaluation(model, x_test, y_test)

In [ ]:
print_confusion_matrix(model, x_test, y_test, labels)

In [ ]:
plot_comparison(history.history['acc'], history.history['val_acc'], "acc")

In [ ]:
plot_comparison(history.history['loss'], history.history['val_loss'], "loss")

### Visualizing activation layers

In [ ]:
def visualize_layers(model, img):
    layer_outputs = [layer.output for layer in model.layers if len(layer.output_shape) > 2 and "dropout" not in layer.name]
    activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
    activations = activation_model.predict(img.reshape((1, img_rows, img_cols, 3)), verbose=0)

    layer_names = []
    images_per_row = 16
    for layer in layer_outputs:
        layer_names.append(layer.name)
        try:
            if layer.filters < images_per_row:
                images_per_row = layer.filters
        except:
            pass
        
    plt.figure(figsize=(1, 1))
    plt.title("Input image", fontsize=10)
    plt.imshow(img)

    for layer_name, layer_activation in zip(layer_names, activations):
        n_features = layer_activation.shape[-1]
        size = layer_activation.shape[1]
        n_cols = n_features // images_per_row
        display_grid = np.zeros((size * n_cols, size * images_per_row))

        for col in range(n_cols):
            for row in range(images_per_row):
                channel_image = layer_activation[0, :, :, col * images_per_row + row]
                channel_image -= channel_image.mean()
                if channel_image.std() != 0:
                    channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image

        scale = 1. / size
        plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')

    plt.show()

In [ ]:
visualize_layers(model, x_test[2])

In [ ]:
visualize_layers(model, x_test[140])

In [ ]:
visualize_layers(model, x_test[36])

### Visualizing filters

In [ ]:
def deprocess_image(x):
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    x += 0.5
    x = np.clip(x, 0, 1)

    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    
    return x

In [ ]:
def generate_pattern(model, layer_name, filter_index, size=150):
    layer_output = model.get_layer(layer_name).output
    loss = K.mean(layer_output[:, :, :, filter_index])

    grads = K.gradients(loss, model.input)[0]
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)
    iterate = K.function([model.input], [loss, grads])
    
    input_img_data = np.random.random((1, size, size, 3)) * 10 + 128.

    step = 1.
    for i in range(40):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step
        
    img = input_img_data[0]
    return deprocess_image(img)

In [ ]:
def rows_cols(value):
    assert value >= 1
    
    rows = 1
    cols = value
    for i in range(2, value//2+1):
        if value % i == 0:
            if np.abs(i - int(value / i)) < np.abs(rows - cols):
                rows = i
                cols = int(value / i)
    
    return rows, cols

In [ ]:
def visualize_filters(model):
    margin = 3
    size = 32
    layers = [layer.name for layer in model.layers if len(layer.output_shape) > 2 and "conv" in layer.name]
    for layer_name in layers: 
        f_count = model.get_layer(layer_name).filters
        rows, cols = rows_cols(f_count)

        results = np.zeros((rows * size + (rows-1) * margin, cols * size + (cols-1) * margin, 3), dtype=np.uint8)
        for i in range(rows):
            for j in range(cols):
                filter_img = generate_pattern(model, layer_name, j + (i * cols), size=size)
                horizontal_start = i * size + i * margin
                horizontal_end = horizontal_start + size
                vertical_start = j * size + j * margin
                vertical_end = vertical_start + size
                results[horizontal_start : horizontal_end, vertical_start : vertical_end, : ] = filter_img

        print("Layer name: " + layer_name)
        plt.figure(figsize=(size / 2, size / 2))
        plt.imshow(results)
        plt.show()

In [ ]:
visualize_filters(model)

### Problem 1

In [ ]:
show_class_example(x_test, y_test, labels, [1, 0])

### Problem 2

In [ ]:
show_class_example(x_test, y_test, labels, [12, 14])

### Problem 3

In [ ]:
show_class_example(x_test, y_test, labels, [14, 17])

### Problem 4

In [ ]:
show_class_example(x_test, y_test, labels, [34, 12])

### Problem 5

In [ ]:
show_class_example(x_test, y_test, labels, [38, 34])

### Problem 6

In [ ]:
show_class_example(x_test, y_test, labels, [35, 33])